In [72]:
import ollama
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "mongodb+srv://<username>:<password>@search.<cluster>.mongodb.net/?retryWrites=true&w=majority&appName=search"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)



Pinged your deployment. You successfully connected to MongoDB!


In [79]:
searchPrompt = "sushi resturants"

textSearch = ollama.embeddings( model='mxbai-embed-large' , prompt=searchPrompt )

print(textSearch["embedding"])
#"Best Date spots"
#"best sushi resturants"
#"where should i go for a business meeting?"
# "bad resturants for my worst enemy"
# "resturants to avoid"

[-0.21447104215621948, 0.6099299788475037, -0.6981145143508911, -0.11895577609539032, -0.42111438512802124, -0.27999892830848694, -0.10185307264328003, 0.580341100692749, 0.49771273136138916, 0.5825477838516235, 0.30853450298309326, 0.43802550435066223, 0.3755151033401489, 0.03837312012910843, -0.5459979772567749, 0.029106322675943375, -0.35679030418395996, 0.052976205945014954, -0.9106940627098083, -0.5609796047210693, 0.30887001752853394, 0.1312182992696762, 0.10908067226409912, -0.9384739995002747, -0.13720403611660004, 0.776644766330719, 0.9335048198699951, 0.26498085260391235, 0.8086845874786377, 0.20367294549942017, -0.478826642036438, 0.08263438940048218, 0.4644966125488281, -0.37816330790519714, 0.3779008388519287, 0.34036117792129517, -0.1606024205684662, -0.14779032766819, -0.15942005813121796, -0.7153427600860596, -0.49167537689208984, -0.8772976398468018, 0.8964411616325378, -0.49211329221725464, -0.12168380618095398, 0.7339696288108826, -0.4602605402469635, -0.743830144405

In [80]:
# THIS WORKS

results = client.WaB.reviewsEmbeded.aggregate([
  {
    "$vectorSearch": {
      "index": "reviews_embed",
      "path": "text_embedding",
      "queryVector": textSearch["embedding"],
      "numCandidates": 10,
      "limit": 10
    }
  },
  {
        '$project': {
            '_id': 0,
            'restaurant_name': 1, 
            'review_text': 1,
            'rating': 1
        }
  }
])


resText = list(results)
# resText = list(results)[0]['review_text']

print(resText)

[{'restaurant_name': 'Sushi World', 'rating': 4, 'review_text': 'Good sushi and nice ambiance. Will definitely come back.'}, {'restaurant_name': 'Sushi World', 'rating': 4, 'review_text': 'Great sushi and friendly staff. Would definitely return.'}, {'restaurant_name': 'Sushi World', 'rating': 4, 'review_text': 'Great sushi and friendly staff. Would definitely return.'}, {'restaurant_name': 'Sushi World', 'rating': 4, 'review_text': 'Great sushi and friendly staff. Would definitely return.'}, {'restaurant_name': 'Sushi World', 'rating': 4, 'review_text': 'Great sushi and friendly staff. Would definitely return.'}, {'restaurant_name': 'Sushi World', 'rating': 4, 'review_text': 'Great sushi and friendly staff. Would definitely return.'}, {'restaurant_name': 'Sushi World', 'rating': 4, 'review_text': 'Fresh and delicious sushi. The rolls were beautifully crafted.'}, {'restaurant_name': 'Sushi World', 'rating': 4, 'review_text': 'Good sushi and nice ambiance. Will definitely come back.'}, {

In [81]:
# an example prompt
prompt = searchPrompt
data = resText

# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama2",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])



Based on the data provided, here are some insights about sushi restaurants:

1. Sushi World is a popular choice among customers, with an average rating of 4 out of 5 stars. Customers appreciate the good sushi and nice ambiance.
2. Tokyo Grill is the highest-rated sushi restaurant in the dataset, with a perfect 5-star rating. Customers praise the fantastic Japanese cuisine, including the teppanyaki and sushi. The minimalist decor and efficient service add to the overall dining experience.
3. All of the restaurants mentioned have an average rating of 4 stars or higher. This suggests that customers generally enjoy their experiences at these sushi restaurants.
4. The review text for each restaurant provides additional insights into the quality of the food and service. For example, one customer praises the "beautifully crafted" rolls at Sushi World, while another mentions the "fresh and delicious" sushi at Tokyo Grill.
5. The lack of diversity in the restaurants listed suggests that there 